# SAE Modélisation Mathématique

MARIE Nathan <br>
COUET Benjamin

### Packages et librairies

In [11]:
%pip install pandas
%pip install tqdm
%pip install spacy
!python3 -m spacy download en_core_web_sm
!python -m spacy download en_core_web_sm
%pip install scikit-learn

import pandas as pd
from tqdm import tqdm
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import spacy
from tqdm import tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 29.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 20.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Note: you may need to restart the kernel to use updated packages.


### Definition des variables globales

In [12]:
BOLD = '\033[1m' # ACTIONS
BLUE = '\033[94m' # ACTIONSv
RESET = '\033[0m'
RED = '\033[91m' # ERRORS
GREEN = '\033[92m' # SUCCESS
YELLOW = '\033[93m' # INFORMATIONS

### Nettoyage des données
Ce nettoyage consiste en .....

In [13]:
def traitementData():
    print(BOLD+BLUE+"\n\nChargement des données...\n\n*")
    df = pd.concat([chunk for chunk in tqdm(pd.read_json('./data/Video_Games.json', lines=True, chunksize=1000), desc=BLUE+'Chargement des données')])
    df = df.drop(['style', 'image'], axis=1)


    df['vote'] = df['vote'].fillna(0.0)

    df['reviewTime'] = pd.to_datetime(df['reviewTime'], format='%m %d, %Y')

    df['year'] = df['reviewTime'].dt.year

    os.makedirs('./split_data', exist_ok=True)

    for year, group in df.groupby('year'):
        if not os.path.exists(f'./split_data/reviews_{year}.json'):
            group.to_json(f'./split_data/reviews_{year}.json', orient='records', lines=True)
            print(GREEN + f'Fichier JSON créé pour l\'année {year}' + RESET)
        else:
            print(GREEN + f'Fichier JSON pour l\'année {year} existe déjà !' + RESET)
        
        group=group.drop_duplicates(subset=["asin","reviewerID","vote"], keep='last', inplace=True)
        if group!=None:
            print(f"{RED}Problème de doublons sur : {group}")
        else:
            print(f"{YELLOW}PAS DE DOUBLONS")


    print(BLUE+'*\n\n'+RESET+BOLD+"Dimensions du DataFrame => "+YELLOW+f"{df.shape}" + RESET)
    print(BOLD+"\nColonnes => "+YELLOW+f"{df.columns.to_list()}"+RESET)
    print(BOLD+"\nNombre de textes d'avis null => "+YELLOW+f"{df['reviewText'].isnull().sum()}"+RESET)
    print(BOLD+"\nNombre de titres d'avis null => "+YELLOW+f"{df['summary'].isnull().sum()}"+RESET)
    print(BOLD+"\nNombre de notes null => "+YELLOW+f"{df['overall'].isnull().sum()}"+RESET)
    print(BOLD+"\nNombre de votes null => "+YELLOW+f"{df['vote'].isnull().sum()}"+RESET)

In [14]:
traitementData()



Chargement des données...

*


Chargement des données: 2566it [00:17, 144.00it/s]


Fichier JSON pour l'année 1997 existe déjà !
PAS DE DOUBLONS
Fichier JSON pour l'année 1998 existe déjà !
PAS DE DOUBLONS
Fichier JSON pour l'année 1999 existe déjà !
PAS DE DOUBLONS
Fichier JSON pour l'année 2000 existe déjà !
PAS DE DOUBLONS
Fichier JSON pour l'année 2001 existe déjà !
PAS DE DOUBLONS
Fichier JSON pour l'année 2002 existe déjà !
PAS DE DOUBLONS
Fichier JSON pour l'année 2003 existe déjà !
PAS DE DOUBLONS
Fichier JSON pour l'année 2004 existe déjà !
PAS DE DOUBLONS
Fichier JSON pour l'année 2005 existe déjà !
PAS DE DOUBLONS
Fichier JSON pour l'année 2006 existe déjà !
PAS DE DOUBLONS
Fichier JSON pour l'année 2007 existe déjà !
PAS DE DOUBLONS
Fichier JSON pour l'année 2008 existe déjà !
PAS DE DOUBLONS
Fichier JSON pour l'année 2009 existe déjà !
PAS DE DOUBLONS
Fichier JSON pour l'année 2010 existe déjà !
PAS DE DOUBLONS
Fichier JSON pour l'année 2011 existe déjà !
PAS DE DOUBLONS
Fichier JSON pour l'année 2012 existe déjà !
PAS DE DOUBLONS
Fichier JSON pour l'anné

### Sélection du jeu de données à utiliser

In [16]:
df=pd.read_json('./split_data/reviews_2000.json', lines=True)

df = df.dropna(subset=['reviewText'])
df = df[df['reviewText'] != '']
df['label'] = df['overall'].apply(lambda x: 1 if x > 3 else 0) 

if not df['reviewText'].notna().all():
    print(RED+"Certains reviewText sont null")
else:
    print(GREEN+"Aucun reviewText null")

Aucun reviewText null
